In [ ]:
from xpeem_utils import *
from skimage.io import imread
import numpy as np
from tifffile import imsave
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200


The timeseries data consists of movies in the .tiff format. Movie 2 goes from 30-48 L of O2 exposure and movie 3 goes from 48-65 L of O2 exposure.  Prior to 30 L of exposure, no islands are observed so they are not included in this demo

In [ ]:
#Read a timeseries data
mov2 = imread('Data/2-CrXAS-movie@577-5eV.tif')
mov3 = imread('Data/3-CrXAS-movie@577-5eV.tif')

#Read Masks
mov2_mask = imread("Data/2nd_oxidation_mask.tif")
mov3_mask = imread("Data/3rd_oxidation_mask.tif")

In [ ]:
#Plot: a) a representative frame from movie 2, b) the mask used for movie 2, 
# c) a representative frame from movie 3, and d) a representative frame from movie 3.
fig, axs = plt.subplots(ncols=4,)
axs[0].imshow(mov2[50,:,:])
axs[0].title.set_text("a)")
axs[1].imshow(mov2_mask)
axs[1].title.set_text("b)")
axs[2].imshow(mov3[50,:,:])
axs[2].title.set_text("c)")
axs[3].imshow(mov3_mask)
axs[3].title.set_text("d)")
for ax in axs:
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)

In [ ]:
#Perform leveling and segmentation of an image, defining the tile size, original image size, and threshold
#the function batch_level_segment() was implemented efficiently using the Numba package
mov2_segmented = batch_level_segment(mov2 ,(64,64),(1024,1024),8000)
mov3_segmented = batch_level_segment(mov3 ,(64,64),(1024,1024),8000)

#Mask the image after leveling and segmenting to avoid plane leveling artifacts at the mask edges
mov2_masked_segmented = mask_subtract_arr(mov2_segmented, mov2_mask)
mov3_masked_segmented = mask_subtract_arr(mov3_segmented, mov3_mask)

#Optionally save the data as a .tif
#imsave("mov2_segmented.tif",mov2_masked_segmented.astype(np.uint8))
#imsave("mov3_segmented.tif",mov3_masked_segmented.astype(np.uint8))

In [ ]:
#Display some representative frames from the segmented images from a) Movie 2 and b) Movie 3
fig, axs = plt.subplots(ncols=2)
axs[0].imshow(mov2_masked_segmented[200,:,:])
axs[1].imshow(mov3_masked_segmented[200,:,:])
for ax in axs:
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)

In [ ]:
#Finds the particles in the segmented timeseries image and puts them into a .csv file 
mov2_particles = findparticles_3d_img(mov2_masked_segmented, minsize=5, maxsize=200)     #minsize of 5 for the crox particles, and 15 for the dark regions
mov3_particles = findparticles_3d_img(mov3_masked_segmented, minsize=5, maxsize=200)

#Optionally save the particle data to a .csv file
# mov2_particles.to_csv("mov2_particles.csv")
# mov3_particles.to_csv("mov3_particles.csv")

Visualize the time-series data.  This will reproduce the components of Figure 10 from the manuscript

In [ ]:
#index the frame numbers to an absolute amount of Oxygen exposure
mov2_particles['langmuir'] = frame_to_langmuir(mov2_particles['frame']+155)
mov3_particles['langmuir'] = frame_to_langmuir(mov3_particles['frame']+450)
print(mov3_particles.head())
print(mov2_particles['langmuir'].max(),mov2_particles['langmuir'].min())
print(mov3_particles['langmuir'].max(),mov3_particles['langmuir'].min())

In [ ]:
#get stats for the particles in movies 2 and 3, and separate out the particles in the upper and lower grains.
#Because of the masks and the alignment process, the position of the grian boundary split is slightly different for movie 2 and 3 

mov2_ug_stats = get_particle_stats(mov2_particles[(mov2_particles['centroid-0'] <600)]) 
mov2_lg_stats = get_particle_stats(mov2_particles[(mov2_particles['centroid-0'] >=600)])
mov3_ug_stats = get_particle_stats(mov3_particles[(mov3_particles['centroid-0'] <560)])
mov3_lg_stats = get_particle_stats(mov3_particles[(mov3_particles['centroid-0'] >=560)])

In [ ]:
#Remove frames that are out of focus
mov2_ug_stats_filtered = mov2_ug_stats[(mov2_ug_stats['langmuir']<=39) | (mov2_ug_stats['langmuir']>=44)]
mov2_lg_stats_filtered = mov2_lg_stats[(mov2_lg_stats['langmuir']<=39) | (mov2_lg_stats['langmuir']>=44)]
mov3_ug_stats_filtered = mov3_ug_stats[(mov3_ug_stats['langmuir']<=59.5) & ~((mov3_ug_stats['langmuir']<=57)& (mov3_ug_stats['langmuir']>=55))]
mov3_lg_stats_filtered = mov3_lg_stats[(mov3_lg_stats['langmuir']<=59.5) & ~((mov3_lg_stats['langmuir']<=57)& (mov3_lg_stats['langmuir']>=55))]
# plt.scatter(mov2_ug_stats_filtered['frame'],mov2_ug_stats_filtered['langmuir'])

In [ ]:
fig, ax = plt.subplots()

ax.scatter(mov2_ug_stats_filtered['langmuir'],mov2_ug_stats_filtered['frequency']/169.6,color='xkcd:cobalt')
ax.scatter(mov2_lg_stats_filtered['langmuir'],mov2_lg_stats_filtered['frequency']/132.1,color='xkcd:lavender')
ax.scatter(mov3_ug_stats_filtered['langmuir'],mov3_ug_stats_filtered['frequency']/153,color='xkcd:cobalt')
ax.scatter(mov3_lg_stats_filtered['langmuir'],mov3_lg_stats_filtered['frequency']/186,color='xkcd:lavender')
#annotate the plot with the densities measured from the data dimensionality reduction analysis of the hyperspectral images
ax.annotate('+', (65, 5.76), color='xkcd:lavender',fontsize=30)
ax.annotate('x', (65, 5.73),color='xkcd:cobalt',fontsize=30)
#plt.annotate('+', (65, 1.94), color='xkcd:lavender',fontsize=30)
ax.legend(["Upper (212) Grain", "Lower (104) Grain"])
# plt.plot(upper_lang, logistic(upper_lang, *popt_upper), color = 'xkcd:cobalt')
# plt.plot(lower_lang, logistic(lower_lang, *popt_lower),color = 'xkcd:lavender')
ax.set_xlim((30,68))
ax.set_ylim((0,8))
ax.set_xlabel("Oxygen exposure (L)")
ax.set_ylabel("Particle density (particles/um$^2$)")
secax = ax.secondary_xaxis('top')
#Movie 2 starts at 1345 seconds, the oxidation ends at 6202 seconds
timerange = np.linspace(1345,6206,8,dtype=int)
secax.set_xticklabels((timerange))
secax.set_xlabel("Exposure time (s)")

In [ ]:

#Plot the median size of the islands in pixels and convert to um^2
plt.scatter(mov2_ug_stats_filtered['langmuir'], mov2_ug_stats_filtered['median_size']*.025**2, color='xkcd:cobalt') #convert pixel scale to um^2 by multiplying by 0.025 um/px
plt.scatter(mov2_lg_stats_filtered['langmuir'], mov2_lg_stats_filtered['median_size']*.025**2,color='xkcd:lavender') 
plt.scatter(mov3_ug_stats_filtered['langmuir'], mov3_ug_stats_filtered['median_size']*.025**2,color='xkcd:lavender')
plt.scatter(mov3_lg_stats_filtered['langmuir'], mov3_lg_stats_filtered['median_size']*.025**2, color='xkcd:cobalt')
#plot from 35-65.  Below 35 segmentation identifies some particles but these are not clearly oxide islands, and have high variability
plt.xlim((35,65))
plt.ylim((0.003,.009))
plt.ylabel("Median island size ($\mu$m$^2$)")
plt.xlabel("Oxygen Exposure (L)")
plt.legend(["Upper (212) grain","Lower (104) grain"])
plt.title("Median island size")